In [ ]:
# I guess it is best to use Julia 1.9 because it works with the packages in my case
# IN GENERAL: Lets try to use self explaining variable names all the time

using Pkg
# you need to create an environment first, then activate it (in this case mine is called "optimization19")
Pkg.activate("1stEnv")
# - - #
# Install all the packages mentioned below via the following commands
# using Pkg
# Pkg.add("Optim") etc.

# Load packages and obtain the solver LICENSE
using Optim, TrafficAssignment, LinearAlgebra, JuMP, DelimitedFiles, PrettyTables, CSV, LsqFit, Printf

#Include the needed functions (alphabetically) - make sure that functions are stored in the right directory
include("moco_functions/daily_allocated_mocos.jl")      # compute amount of daily mocos (MobilityCoins) needed based on maintaining traffic patterns without       
include("moco_functions/allocated_mocos.jl")     # compute amount of total mocos (MobilityCoins) needed based on maintaining traffic patterns without coins
include("moco_functions/daily_consumed_mocos.jl")       # amount of mocos consumed per day based on the last traffic assignment
include("moco_functions/consumed_mocos.jl")             # amount of mocos consumed at the end of the considered period 
include("moco_functions/emissions.jl")
include("moco_functions/load_network.jl")               # load network demand and supply (status quo "Visum Munich Transportation Model")
include("moco_functions/moco_link_costs_wo_mp.jl")      # compute the link costs without market price
include("moco_functions/mode_choice_tt.jl")             # compute mode choice model based on current travel times
include("moco_functions/shortest_path.jl")              # compute shortest paths OD matrix
include("moco_functions/shortest_traveltime.jl")        # compute shortest travel time for mode choice function 
include("moco_functions/transportation_model.jl")       # includes mode choice and traffic assignment
println("all functions included")


# Load the SiouxFalls network from the TrafficAssignment repository. We will use the SiouxFalls network for development and debugging
ta_data = load_ta_network("SiouxFalls")
ta_datac = load_ta_network("SiouxFalls")
ta_datab = load_ta_network("SiouxFalls")
ta_datar = load_ta_network("SiouxFalls")
# We might use the network below later. Could be challenging to make it work
# ta_data = load_ta_network("ring24total")
# ta_datac = load_ta_network("ring24c")
# ta_datab = load_ta_network("ring24b")
# ta_datar = load_ta_network("ring24r")


sqcardemand = ta_datac.travel_demand 
sqbusdemand = ta_datab.travel_demand 
sqraddemand = ta_datar.travel_demand 

println("car demand status quo = ",sum(sqcardemand))
println("bus demand status quo = ",sum(sqbusdemand))
println("bike demand status quo = ",sum(sqraddemand))

In [ ]:
# SETTING UP MOCO EMISSION AND INCENTIVE PARAMETER
# specific avg externalities/incentives per mode
# https://www.nachhaltigkeitsstrategie.de/strategie/projekte/detail-1/mit-dem-oepnv-das-klima-schuetzen?utm_source=chatgpt.com
# There’s no official public number for CO₂/pkm specifically for the MVV region. However, MVV roughly reflects the German averages:
# Public transport around 65–75 g/pkm Car around 150 g/pkm Cycling/walking essentially zero CO₂.
# Additionally, on average: Cars cause 147 g/pkm (assuming 1.5 passengers), Public transport (bus/train) averages 65 g/pkm
# emissions ------------------------------
carext = 0.147     # kgCO2 per Pkm
busext = 0.065     # kgCO2 per Pkm 
# bikeext = 0      # kgCO2 per Pkm
# ----------------------------------------

# moco price per emission ----------------
mocperkgCO2 = 1   # MoCo per kg CO2 for the start
# ----------------------------------------

# bike incentive -------------------------
bikeincentive = 0 # -> no incentive for the start
# ----------------------------------------

# mocomp ---------------------------------
mocomp = 1 # market price equals one at the start
# ---------------------------------------------------------------------------

println("done 1/10")


# LOAD NETWORK DATA >>> function load_network(basenet, carnet, busnet, bikenet)
nodes, nnodes, zones, nzones, n, arcs, demandc, demandb, demandr, totaldemand, destinations, arcdest, demand_dict, isod, bpr = @time load_network(ta_data, ta_datac, ta_datab, ta_datar)
println("done 2/10")



# first traffic assignment car w/o moco -> output: link travel times car
link_flowc, link_travel_timecsq, objectivec =
@time ta_frank_wolfe(ta_datac, method=:cfw, max_iter_no=50, step=:exact, log=:off, tol=1e-1)

# link travel times bus ("linearly" congested)
ta_datab.power .= 1                                     
link_travel_timebsq = ta_datab.free_flow_time # .+ 1/60 * 5 # 5 minute waiting time at the bus stop included in mode-choice

# link travel times bike ("linearly" congested)
ta_datar.power .= 1                                     
link_travel_timersq = ta_datar.link_length / 15  # 15km/h avg bike velocity
ta_datar.capacity .= 100
println("done 3/10")



# COMPUTE SHORTEST PATHS
dist = @time shortest_path(ta_data)
println("done 4/10")

# COMPUTE SHORTEST TRAVELTIMES 
ttsc, ttsb, ttsr = @time shortest_traveltime(ta_data, link_travel_timecsq, link_travel_timebsq, link_travel_timersq)
println("done 5/10")

# COMPUTE MOCO COSTS WITHOUT MARKET PRICE
mpdictwomp = moco_link_costs_wo_mp(dist, carext, mocperkgCO2)

println("done 6/10")

# ENABLE TOLL BPR TERM -> that is activating the toll term of the TrafficAssignment package. we use it for the mobilitycoin charges
ta_datac.toll_factor = 1
ta_datab.toll_factor = 1
ta_datar.toll_factor = 1
println("done 7/10")

# pluggin in VOT via free flow travel time
# that is important because we are using the TrafficAssignment package which is why I dont change the cost function itself
ta_datac.free_flow_time = ta_datac.free_flow_time .* 30 # 30 EUR VOT for car passengers
ta_datab.free_flow_time = ta_datab.free_flow_time .* 20 # 20 EUR VOT for bus passengers
ta_datar.free_flow_time = ta_datar.free_flow_time .* 20 # 20 EUR VOT for cyclists
println("done 8/10")

# prepare bike links for car and bus mode (the following code doesnt apply to the SiouxFalls case)
# Iterate over the range from 1 to number_of_links to adjust the capacity of bike links for mode car
for i in 1:ta_datac.number_of_links
    # Check if the link_type is 10 (bike link)
    if ta_datac.link_type[i] == 10
        # Adjust the capacity to 0.00001
        ta_datac.free_flow_time[i] = 100
    end
end

for i in 1:ta_datab.number_of_links
    # Check if the link_type is 10 (bike link)
    if ta_datab.link_type[i] == 10
        # Adjust the capacity to 0.00001
        ta_datab.free_flow_time[i] = 100
    end
end
println("done 9/10")


# Store status quo link travel times in a CSV file
dftt = DataFrame(link_travel_timec = link_travel_timecsq,
link_travel_timeb = link_travel_timebsq,
link_travel_timer = link_travel_timersq)
# Write the DataFrame to a CSV file
CSV.write("traveltimes.csv", dftt)

println("done 10/10")

In [ ]:
# preparing the arrays for recording results
mocperkgCO2arr = []
MParr = []
coinsallocatedarr = []
coinsneededarr = []
deltaarr = []
mocarr = []
cardemarr = []
busdemarr = []
bikedemarr = []
flowscararr = []
flowsbusarr = []
flowsbikearr = []
mocoscararr = []
mocosbusarr = []
mocosbikearr = []
emissionscararr = []
emissionsbusarr = []
totaltt = []
totalttc = []
totalttb = []
totalttr = []
totaldcarr = []
totaldbarr = []
totaldrarr = []
#row_index_arr = []  # New array to track row index

# adding headers to the arrays
push!(mocperkgCO2arr, "mocperkgCO2")
push!(MParr, "mocomp")
push!(coinsallocatedarr, "coins allocated")
push!(coinsneededarr, "coins consumed")
push!(deltaarr, "market delta")
push!(cardemarr, "cardem")
push!(busdemarr, "busdem")
push!(bikedemarr, "bikedem")
push!(flowscararr, "flows car")
push!(flowsbusarr, "flows bus")
push!(flowsbikearr, "flows bike")
push!(mocoscararr, "mocoscar needed")
push!(mocosbusarr, "mocosbus needed")
push!(mocosbikearr, "mocosbike needed")
push!(emissionscararr, "car emissions")
push!(emissionsbusarr, "bus emissions")
push!(totaltt, "total_tt")
push!(totalttc, "total_tt_c")
push!(totalttb, "total_tt_b")
push!(totalttr, "total_tt_r")
push!(totaldcarr, "total dist car")
push!(totaldbarr, "total dist bus")
push!(totaldrarr, "total dist rad")
#push!(row_index_arr, "Row Index")  # Header for row index

# Row counter
row_index = 1  # Start at row 1 after the headers

function roundarray(arr)
    return [ (x isa Number && !isa(x, Int)) ? round(x, digits=2) : x for x in arr ]
end

# definition of objective function for brent algorithm
function objective_function_1(initialMP, mocperkgCO2)
    global row_index  # Ensure row_index is modified inside function
    mocomp = initialMP #[1]
    
    
    coinsneeded, 
    cardem, busdem, bikedem, 
    mocoscar, mocosbus, mocosbike, 
    flowscar, flowsbus, flowsbike, 
    totalemissions, caremissions, busemissions, 
    total_tt, total_tt_c, total_tt_b, total_tt_r, 
    link_travel_timec, link_travel_timeb, link_travel_timer, 
    totaldistcar, totaldistbus, totaldistrad = transportation_model(
        ta_data, 
        mocomp, 
        0,                      # third argument = bikeincentive = 0
        mocperkgCO2, 
        ttsc, 
        ttsb, 
        ttsr
    ) 
    



    # Objective is to minimize the absolute difference between coinsallocated and coinsneeded
    delta = abs(coinsallocated - coinsneeded)
    
    push!(mocperkgCO2arr, mocperkgCO2)
    push!(MParr, mocomp)
    push!(coinsallocatedarr, coinsallocated)
    push!(coinsneededarr, coinsneeded)
    push!(deltaarr, delta)
    push!(cardemarr, cardem)
    push!(busdemarr, busdem)
    push!(bikedemarr, bikedem)
    push!(flowscararr, flowscar)
    push!(flowsbusarr, flowsbus)
    push!(flowsbikearr, flowsbike)
    push!(mocoscararr, mocoscar)
    push!(mocosbusarr, mocosbus)
    push!(mocosbikearr, mocosbike)
    push!(emissionscararr, caremissions)
    push!(emissionsbusarr, busemissions)
    push!(totaltt, total_tt)
    push!(totalttc, total_tt_c)
    push!(totalttb, total_tt_b)
    push!(totalttr, total_tt_r)
    push!(totaldcarr, totaldistcar)
    push!(totaldbarr, totaldistbus)
    push!(totaldrarr, totaldistrad)

# Round arrays individually
rounded_columns = [
    roundarray(mocperkgCO2arr),
    roundarray(MParr),
    roundarray(coinsallocatedarr),
    roundarray(coinsneededarr),
    roundarray(deltaarr),
    roundarray(cardemarr),
    roundarray(busdemarr),
    roundarray(bikedemarr),
    roundarray(flowscararr),
    roundarray(flowsbusarr),
    roundarray(flowsbikearr),
    roundarray(mocoscararr),
    roundarray(mocosbusarr),
    roundarray(mocosbikearr),
    roundarray(emissionscararr),
    roundarray(emissionsbusarr),
    roundarray(totaltt),
    roundarray(totalttc),
    roundarray(totalttb),
    roundarray(totalttr),
    roundarray(totaldcarr),
    roundarray(totaldbarr),
    roundarray(totaldrarr)
]

result_matrix = hcat(rounded_columns...)
writedlm("results.csv", result_matrix, ',')
    return (delta)
end

In [ ]:
# in case we change something, we can reload the transportation model
include("moco_functions/transportation_model.jl")       # includes mode choice and traffic assignment

In [ ]:
# in case we change something we can reload the mode-choice model
include("moco_functions/mode_choice_tt.jl")             # compute mode choice model based on current travel times

In [ ]:
coinsallocated = 655040 #655030 #655030 #645860 # 1.1369356986281308e6 # 1.164475393364052e6

mocperkgCO2 = 1 # 1.05
initialMP = 1

# in case we change the borders for the solution space
# borders = [0.0,
#            0.1,
#            0.2,
#            0.5,
#            1.0,
#            1.5,
#            2.0,
#            2.5,
#            3.0]#,
    #    1.4,
    #    1.6,
    #    1.8,
    #    2.0,
    #    2.2,
    #    2.4,
    #    2.6,
    #    2.8,
    #    3.0] #16

# in case we change the allocated coins
# coinsallocatedarr1 = [770000]

mocperkg = [1.0]
        # 2.5,
        # 5.0,
        # 7.5]
final_result_row = ()

#for coinsallocated in coinsallocatedarr1
for mocperkgCO2 in mocperkg
    
    local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
    # Optimizing the objective function
    # You can choose different methods available in Optim, here BFGS is used as an example
    result = optimize(local_objective, 0.0, 10.0, Brent();
                            iterations = 15,
                            store_trace = true,
                            show_trace = true
                            )


    # Extracting the optimized value of mocomp
    optimized_mocomp = Optim.minimizer(result)
    optimized_value = Optim.minimum(result)
    #iter = iterations(result)
    #traceres = trace(result)
    # Find the row where delta is minimized
    min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
    println("Optimization happened at row: ", min_delta_index) 

    # Save the row of minimum delta in a separate file
    final_result_row = ( 
        mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
        mocomp = MParr[min_delta_index], 
        coins_allocated = coinsallocatedarr[min_delta_index], 
        coins_needed = coinsneededarr[min_delta_index], 
        market_delta = deltaarr[min_delta_index], 
        cardem = cardemarr[min_delta_index], 
        busdem = busdemarr[min_delta_index], 
        bikedem = bikedemarr[min_delta_index], 
        flows_car = flowscararr[min_delta_index], 
        flows_bus = flowsbusarr[min_delta_index], 
        flows_bike = flowsbikearr[min_delta_index], 
        mocos_car = mocoscararr[min_delta_index], 
        mocos_bus = mocosbusarr[min_delta_index], 
        mocos_bike = mocosbikearr[min_delta_index], 
        emissions_car = emissionscararr[min_delta_index], 
        emissions_bus = emissionsbusarr[min_delta_index], 
        total_tt = totaltt[min_delta_index], 
        total_tt_c = totalttc[min_delta_index], 
        total_tt_b = totalttb[min_delta_index], 
        total_tt_r = totalttr[min_delta_index], 
        total_dist_car = totaldcarr[min_delta_index], 
        total_dist_bus = totaldbarr[min_delta_index], 
        total_dist_rad = totaldrarr[min_delta_index]
    )
   
    # Write header and data to CSV
    writedlm("Final_Result.csv", final_result_row, ',')
    
end
println(final_result_row)

In [ ]:
# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount

# Define the generalized logistic function
logistic_model(x, p) = p[1] ./ (1 .+ exp.(-p[2] .* (x .- p[3])))

# Initial guess for parameters [L, k, x0]
p0 = [1.0, 0.5, 10.0]

# Fit the model to the data
fit = curve_fit(logistic_model, x_data, y_data, p0)

# Extract estimated parameters
L_fit, k_fit, x0_fit = fit.param

println("Estimated parameters:")
println("L = $(round(L_fit, digits=6)), k = $(round(k_fit, digits=6)), x0 = $(round(x0_fit, digits=6))")
println("Best-fit equation: y = $L_fit / (1 + exp(-$k_fit * (x - $x0_fit)))")

# Compute model values for x = 1 to 20
x_values = 1:20
y_values = logistic_model(x_values, fit.param)

# Print the results
println("\nModel Predictions:")
for (x, y) in zip(x_values, y_values)
    println("x = $(lpad(x, 2)) → y = $(round(y, digits=6))")
end

# Calculate and print the sum of all y-values
total_y = sum(y_values)
println("\nSum of all predicted y-values from x = 1 to 20: $(round(total_y, digits=6))")

In [ ]:
using LsqFit

# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount

# Define the generalized logistic function
logistic_model(x, p) = p[1] ./ (1 .+ exp.(-p[2] .* (x .- p[3])))

# Initial guess for parameters [L, k, x0]
p0 = [1.0, 0.5, 10.0]

# Fit the model to the data
fit = curve_fit(logistic_model, x_data, y_data, p0)

# Extract estimated parameters
L_fit, k_fit, x0_fit = fit.param

# Round parameters for display
L_fit_rnd = round(L_fit, digits=2)
k_fit_rnd = round(k_fit, digits=2)
x0_fit_rnd = round(x0_fit, digits=2)

println("Estimated parameters (rounded to 2 decimals):")
println("L = $L_fit_rnd, k = $k_fit_rnd, x0 = $x0_fit_rnd")

println("Best-fit equation: y = $L_fit_rnd / (1 + exp(-$k_fit_rnd * (x - $x0_fit_rnd)))")

# Compute model values for x = 1 to 20
x_values = 1:20
y_values = logistic_model(x_values, fit.param)

# Print rounded results
println("\nModel Predictions (rounded to 2 decimals):")
for (x, y) in zip(x_values, y_values)
    println("x = $(lpad(x, 3)) → y = $(round(y, digits=3))")
end

# Calculate and print the sum of all y-values, rounded
total_y = sum(y_values)
println("\nSum of all predicted y-values from x = 1 to 20: $(round(total_y, digits=3))")

In [ ]:
using LsqFit
using Plots

# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount (in fraction)

# Define the generalized logistic function
logistic_model(x, p) = p[1] ./ (1 .+ exp.(-p[2] .* (x .- p[3])))

# Initial guess for parameters [L, k, x0]
p0 = [1.0, 0.5, 10.0]

# Fit the model to the data
fit = curve_fit(logistic_model, x_data, y_data, p0)

# Extract estimated parameters
L_fit, k_fit, x0_fit = fit.param

# Round parameters for display
L_fit_rnd = round(L_fit, digits=2)
k_fit_rnd = round(k_fit, digits=2)
x0_fit_rnd = round(x0_fit, digits=2)

println("Estimated parameters (rounded to 2 decimals):")
println("L = $L_fit_rnd, k = $k_fit_rnd, x0 = $x0_fit_rnd")

println("Best-fit equation: y = $L_fit_rnd / (1 + exp(-$k_fit_rnd * (x - $x0_fit_rnd)))")

# Compute model values for x = 1 to 20
x_values = 1:20
y_values_frac = logistic_model(x_values, fit.param)

# Convert to percentage
y_values_pct = y_values_frac .* 100
y_data_pct = y_data .* 100

# Print rounded results
println("\nModel Predictions (rounded to 2 decimals):")
for (x, y) in zip(x_values, y_values_pct)
    println("x = $(lpad(x, 3)) → y = $(round(y, digits=2)) %")
end

# Calculate and print the sum of all y-values (fraction)
total_y = sum(y_values_frac)
println("\nSum of all predicted y-values from x = 1 to 20 (fraction): $(round(total_y, digits=3))")

# Create tick labels:
ytick_positions = 0:1:100
ytick_labels = [mod(y,10)==0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Plot the fitted curve and original data
plt = plot(
    x_values,
    y_values_pct,
    label = "Fitted Logistic Curve",
    xlabel = "Remaining Working Days of Designated Period (Month)",
    ylabel = "Crowdfunded MobilityCoins of Users' Remaining Budget (%)",
    lw = 2,
    legend = :topright,
    title = "Logistic Fit to Individuals' Crowdfunding Willingness",
    xticks = 1:1:20,
    yticks = yticks,
    ylims = (0, 100),
    minorgrid = true,
    xflip = true,
    size = (750, 650),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern"
)

scatter!(
    plt,
    x_data,
    y_data_pct,
    label = "Surveyed Data",
    marker = (:circle, 8),
    color = :red,
)

# Save plot as PNG
savefig(plt, "logistic_fit_crowdfunding_percentage100.png")
println("Plot saved as logistic_fit_crowdfunding_percentage.png")

In [ ]:
using LsqFit
using Plots

# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount (in fraction)

# Solve for exact parameters a and b using two points
b_fit = log(y_data[3] / y_data[1]) / (x_data[3] - x_data[1])
a_fit = y_data[1] / exp(b_fit * x_data[1])

# Round parameters for display
a_fit_rnd = round(a_fit, digits=2)
b_fit_rnd = round(b_fit, digits=2)

println("Exact parameters (rounded to 2 decimals): a = $a_fit_rnd, b = $b_fit_rnd")
println("Best-fit equation: y = $a_fit_rnd * exp($b_fit_rnd * x)")

# Define the exponential function
exp_model(x) = a_fit * exp(b_fit * x)

# Compute model values for x = 1 to 20
x_values = 1:20
y_values_func = exp_model.(x_values)

# Clamp values between 0 and 1
y_values_func = clamp.(y_values_func, 0.0, 1.0)

# Convert to percentage
y_values_perc = y_values_func .* 100
y_data_perc = y_data .* 100

# Print rounded predictions
println("\nModel Predictions (rounded to 3 decimals):")
for (x, y) in zip(x_values, y_values_perc)
    println("x = $(lpad(x, 3)) → y = $(round(y, digits=3)) %")
end

# Calculate and print the sum of all y-values (fraction)
total_y = sum(y_values_func)
println("\nSum of all predicted y-values from x = 1 to 20 (fraction): $(round(total_y, digits=3))")

# Create tick labels:
ytick_positions = 0:1:100
ytick_labels = [mod(y,10)==0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Plot the fitted curve and original data
plt2 = plot(
    x_values,
    y_values_perc,
    label = "Exponential Model Curve",
    xlabel = "Remaining Working Days of Designated Period (Month)",
    ylabel = "Crowdfunded MobilityCoins of Users' Remaining Budget (%)",
    lw = 2,
    legend = :topright,
    title = "Exponential Fit to Individuals' Crowdfunding Willingness",
    xticks = 1:1:20,
    yticks = yticks,
    ylims = (0, 100),
    minorgrid = true,
    xflip = true,
    size = (750, 650),
    titlefontsize = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern"
)

scatter!(
    plt2,
    x_data,
    y_data_perc,
    label = "Surveyed Data",
    marker = (:circle, 8),
    color = :red,
)

# Save plot as PNG
savefig(plt2, "exponential_fit_crowdfunding100.png")
println("Plot saved as exponential_fit_crowdfunding.png")

In [ ]:
using LsqFit
using Plots

# ==========================================
# Data (same for both models)
# ==========================================
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount (fraction)

# ==========================================
# Exponential Fit
# ==========================================
# Solve for exact parameters a and b using two points
b_fit = log(y_data[3] / y_data[1]) / (x_data[3] - x_data[1])
a_fit = y_data[1] / exp(b_fit * x_data[1])

# Exponential model function
exp_model(x) = a_fit * exp(b_fit * x)

# Compute predictions
x_values = 1:20
y_exp = exp_model.(x_values)

# Clamp between 0 and 1
y_exp = clamp.(y_exp, 0.0, 1.0)

# Convert to percentages
y_exp_pct = y_exp .* 100

# ==========================================
# Logistic Fit
# ==========================================
# Generalized logistic function
logistic_model(x, p) = p[1] ./ (1 .+ exp.(-p[2] .* (x .- p[3])))

# Initial guess
p0 = [1.0, 0.5, 10.0]

# Curve fit
fit = curve_fit(logistic_model, x_data, y_data, p0)

L_fit, k_fit, x0_fit = fit.param

# Compute predictions
y_logistic = logistic_model(x_values, fit.param)

# Convert to percentages
y_logistic_pct = y_logistic .* 100

# ==========================================
# Prepare ticks
# ==========================================
ytick_positions = 0:1:100
ytick_labels = [mod(y,10)==0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# ==========================================
# Plot both fits and data points
# ==========================================
plt_combined = plot(
    x_values,
    y_exp_pct,
    label = "Exponential Model Curve",
    lw = 2,
    color = :blue,
    xlabel = "Remaining Working Days of Designated Period (Month)",
    ylabel = "Crowdfunded MobilityCoins of Users' Remaining Budget (%)",
    legend = :topright,
    title = "",
    xticks = 1:1:20,
    yticks = yticks,
    ylims = (0, 100),
    minorgrid = true,
    xflip = true,
    size = (750, 650),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black
)

# Add logistic curve
plot!(
    plt_combined,
    x_values,
    y_logistic_pct,
    label = "Logistic Model Curve",
    lw = 2,
    color = :green,
)

# Add data points
scatter!(
    plt_combined,
    x_data,
    y_data .* 100,
    label = "Surveyed Data",
    marker = (:circle, 8),
    color = :red,
)

# Save combined plot
savefig(plt_combined, "combined_fit_crowdfunding.png")
println("Combined plot saved as combined_fit_crowdfunding.png")

# Optional: display in a window if running interactively
# display(plt_combined)


In [ ]:
x = 1 # x defines number of working day in a month

coinsallocated = 655040 #645860 # 1.1369356986281308e6 # 1.164475393364052e6
allocated_mocos = 0
consuming_mocos = 0

crowdfunding = 0
totalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

totalcardem = 0
totalbusdem = 0
totalbikedem = 0
totaldem = 0

monthly_mocperkgCO2 = 0
last_mocomp = 0
monthly_coins_allocated = 0
monthly_coins_needed = 0
monthly_market_delta = 0
last_cardem = 0
last_busdem = 0
last_bikedem = 0
last_flows_car = 0
last_flows_bus = 0
last_flows_bike = 0
monthly_mocos_car = 0
monthly_mocos_bus = 0
monthly_mocos_bike = 0
monthly_emissions_car = 0
monthly_emissions_bus = 0
monthly_total_tt = 0
monthly_total_tt_c = 0 
monthly_total_tt_b = 0
monthly_total_tt_r = 0
monthly_total_dist_car = 0
monthly_total_dist_bus = 0
monthly_total_dist_rad = 0

mocperkg = [1.0]
        # 2.5,
        # 5.0,
        # 7.5]

monthly_result_row = ()
final_result_row = ()

for x in 1:20 # 20 working days in a month
    #for coinsallocated in coinsallocatedarr1
    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 20,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        final_result_row = ( 
            mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
            mocomp = MParr[min_delta_index], 
            coins_allocated = coinsallocatedarr[min_delta_index], 
            coins_needed = coinsneededarr[min_delta_index], 
            market_delta = deltaarr[min_delta_index], 
            cardem = cardemarr[min_delta_index], 
            busdem = busdemarr[min_delta_index], 
            bikedem = bikedemarr[min_delta_index], 
            flows_car = flowscararr[min_delta_index], 
            flows_bus = flowsbusarr[min_delta_index], 
            flows_bike = flowsbikearr[min_delta_index], 
            mocos_car = mocoscararr[min_delta_index], 
            mocos_bus = mocosbusarr[min_delta_index], 
            mocos_bike = mocosbikearr[min_delta_index], 
            emissions_car = emissionscararr[min_delta_index], 
            emissions_bus = emissionsbusarr[min_delta_index], 
            total_tt = totaltt[min_delta_index], 
            total_tt_c = totalttc[min_delta_index], 
            total_tt_b = totalttb[min_delta_index], 
            total_tt_r = totalttr[min_delta_index], 
            total_dist_car = totaldcarr[min_delta_index], 
            total_dist_bus = totaldbarr[min_delta_index], 
            total_dist_rad = totaldrarr[min_delta_index]
        )
    end

    allocated_mocos += final_result_row.coins_allocated
    consuming_mocos += final_result_row.coins_needed   

    available_allocated_mocos = allocated_mocos #+ crowdfunding
    available_consuming_mocos = consuming_mocos #+ crowdfunding

    nextcardem = final_result_row.cardem
    nextbusdem = final_result_row.busdem
    nextbikedem = final_result_row.bikedem
    nextdem = final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem

    totalcardem += final_result_row.cardem
    totalbusdem += final_result_row.busdem
    totalbikedem += final_result_row.bikedem
    totaldem += (final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem)

    monthly_mocperkgCO2 += final_result_row.mocperkgCO2
    last_mocomp = final_result_row.mocomp
    available_monthly_coins_allocated = available_allocated_mocos
    available_monthly_coins_needed = available_consuming_mocos 
    monthly_market_delta += final_result_row.market_delta
    last_cardem = final_result_row.cardem 
    last_busdem = final_result_row.busdem
    last_bikedem = final_result_row.bikedem
    last_flows_car = final_result_row.flows_car
    last_flows_bus = final_result_row.flows_bus
    last_flows_bike = final_result_row.flows_bike
    monthly_mocos_car += final_result_row.mocos_car
    monthly_mocos_bus += final_result_row.mocos_bus
    monthly_mocos_bike += final_result_row.mocos_bike
    monthly_emissions_car += final_result_row.emissions_car
    monthly_emissions_bus += final_result_row.emissions_bus
    monthly_total_tt += final_result_row.total_tt
    monthly_total_tt_c += final_result_row.total_tt_c 
    monthly_total_tt_b += final_result_row.total_tt_b
    monthly_total_tt_r += final_result_row.total_tt_r
    monthly_total_dist_car += final_result_row.total_dist_car
    monthly_total_dist_bus += final_result_row.total_dist_bus
    monthly_total_dist_rad += final_result_row.total_dist_rad
    first_available_allocated_mocos = available_allocated_mocos
    first_available_consuming_mocos = available_consuming_mocos
    first_crowdfunding = crowdfunding
    monthly_crowdfunding = totalcrowdfunding

    monthly_result_row = ( 
        monthly_mocperkgCO2 = monthly_mocperkgCO2, last_mocomp, available_monthly_coins_allocated, available_monthly_coins_needed, monthly_market_delta,
        last_cardem, last_busdem, last_bikedem, 
        last_flows_car, last_flows_bus, last_flows_bike, 
        monthly_mocos_car, monthly_mocos_bus, monthly_mocos_bike, monthly_emissions_car, monthly_emissions_bus, 
        monthly_total_tt, monthly_total_tt_c, monthly_total_tt_b, monthly_total_tt_r, 
        monthly_total_dist_car, monthly_total_dist_bus, monthly_total_dist_rad,
        totalcardem, totalbusdem, totalbikedem, totaldem, first_crowdfunding, monthly_crowdfunding,
        first_available_allocated_mocos, first_available_consuming_mocos
    )

    coinsallocated = final_result_row.coins_allocated

    mocperkgCO2 = final_result_row.mocperkgCO2
    initialMP = final_result_row.mocomp

    # Write header and data to CSV
    writedlm("monthly_result.csv", monthly_result_row, ',')
    
end
println(monthly_result_row)

In [ ]:
x = 20 # x defines number of working day in a month

coinsallocated = 655040 #670250 #655030 #645860 # 1.1369356986281308e6 # 1.164475393364052e6
allocated_mocos = 0
consuming_mocos = 0

crowdfunding = 0
totalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

totalcardem = 0
totalbusdem = 0
totalbikedem = 0
totaldem = 0

monthly_mocperkgCO2 = 0
last_mocomp = 0
monthly_coins_allocated = 0
monthly_coins_needed = 0
monthly_market_delta = 0
last_cardem = 0
last_busdem = 0
last_bikedem = 0
last_flows_car = 0
last_flows_bus = 0
last_flows_bike = 0
monthly_mocos_car = 0
monthly_mocos_bus = 0
monthly_mocos_bike = 0
monthly_emissions_car = 0
monthly_emissions_bus = 0
monthly_total_tt = 0
monthly_total_tt_c = 0 
monthly_total_tt_b = 0
monthly_total_tt_r = 0
monthly_total_dist_car = 0
monthly_total_dist_bus = 0
monthly_total_dist_rad = 0

mocperkg = [1.0]
        # 2.5,
        # 5.0,
        # 7.5]

monthly_result_row = ()
day_result_row = ()
final_result_row = ()
daily_results_df = DataFrame()
monthly_result_df= DataFrame()

for x in 20:-1:1 # 20 working days in a month
    #for coinsallocated in coinsallocatedarr1
    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        day_result_row = ( 
            coins_allocated = coinsallocatedarr[min_delta_index],
            coins_needed = coinsneededarr[min_delta_index]
        )
    end

    allocated_mocos += day_result_row.coins_allocated
    
    exp_model(x) = 897.27 / (1 + exp(0.22 * (x + 32.69))) # 20 working days in a month
    crowdfunding = exp_model(x) * (655040)
    #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
    totalcrowdfunding += crowdfunding

    coinsallocated = day_result_row.coins_allocated #- crowdfunding

    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        final_result_row = ( 
            mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
            mocomp = MParr[min_delta_index], 
            coins_allocated = coinsallocatedarr[min_delta_index], 
            coins_needed = coinsneededarr[min_delta_index], 
            market_delta = deltaarr[min_delta_index], 
            cardem = cardemarr[min_delta_index], 
            busdem = busdemarr[min_delta_index], 
            bikedem = bikedemarr[min_delta_index], 
            flows_car = flowscararr[min_delta_index], 
            flows_bus = flowsbusarr[min_delta_index], 
            flows_bike = flowsbikearr[min_delta_index], 
            mocos_car = mocoscararr[min_delta_index], 
            mocos_bus = mocosbusarr[min_delta_index], 
            mocos_bike = mocosbikearr[min_delta_index], 
            emissions_car = emissionscararr[min_delta_index], 
            emissions_bus = emissionsbusarr[min_delta_index], 
            total_tt = totaltt[min_delta_index], 
            total_tt_c = totalttc[min_delta_index], 
            total_tt_b = totalttb[min_delta_index], 
            total_tt_r = totalttr[min_delta_index], 
            total_dist_car = totaldcarr[min_delta_index], 
            total_dist_bus = totaldbarr[min_delta_index], 
            total_dist_rad = totaldrarr[min_delta_index]
        )
    end
    # Merge crowdfunding into your daily row
    daily_row_with_crowdfunding = merge(final_result_row, (crowdfunding = crowdfunding,))
    daily_results_df = DataFrame([daily_row_with_crowdfunding])
    push!(monthly_result_df, daily_results_df[1, :])

    consuming_mocos += final_result_row.coins_needed

    available_allocated_mocos = allocated_mocos #+ crowdfunding
    available_consuming_mocos = consuming_mocos #+ crowdfunding

    nextcardem = final_result_row.cardem
    nextbusdem = final_result_row.busdem
    nextbikedem = final_result_row.bikedem
    nextdem = final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem

    totalcardem += final_result_row.cardem
    totalbusdem += final_result_row.busdem
    totalbikedem += final_result_row.bikedem
    totaldem += (final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem)

    monthly_mocperkgCO2 += final_result_row.mocperkgCO2
    last_mocomp = final_result_row.mocomp
    available_monthly_coins_allocated = available_allocated_mocos
    available_monthly_coins_needed = available_consuming_mocos 
    monthly_market_delta += final_result_row.market_delta
    last_cardem = final_result_row.cardem 
    last_busdem = final_result_row.busdem
    last_bikedem = final_result_row.bikedem
    last_flows_car = final_result_row.flows_car
    last_flows_bus = final_result_row.flows_bus
    last_flows_bike = final_result_row.flows_bike
    monthly_mocos_car += final_result_row.mocos_car
    monthly_mocos_bus += final_result_row.mocos_bus
    monthly_mocos_bike += final_result_row.mocos_bike
    monthly_emissions_car += final_result_row.emissions_car
    monthly_emissions_bus += final_result_row.emissions_bus
    monthly_total_tt += final_result_row.total_tt
    monthly_total_tt_c += final_result_row.total_tt_c 
    monthly_total_tt_b += final_result_row.total_tt_b
    monthly_total_tt_r += final_result_row.total_tt_r
    monthly_total_dist_car += final_result_row.total_dist_car
    monthly_total_dist_bus += final_result_row.total_dist_bus
    monthly_total_dist_rad += final_result_row.total_dist_rad
    first_available_allocated_mocos = available_allocated_mocos
    first_available_consuming_mocos = available_consuming_mocos
    first_crowdfunding = crowdfunding
    monthly_crowdfunding = totalcrowdfunding

    monthly_result_row = ( 
        monthly_mocperkgCO2 = monthly_mocperkgCO2, last_mocomp, available_monthly_coins_allocated, available_monthly_coins_needed, monthly_market_delta,
        last_cardem, last_busdem, last_bikedem, 
        last_flows_car, last_flows_bus, last_flows_bike, 
        monthly_mocos_car, monthly_mocos_bus, monthly_mocos_bike, monthly_emissions_car, monthly_emissions_bus, 
        monthly_total_tt, monthly_total_tt_c, monthly_total_tt_b, monthly_total_tt_r, 
        monthly_total_dist_car, monthly_total_dist_bus, monthly_total_dist_rad,
        totalcardem, totalbusdem, totalbikedem, totaldem, first_crowdfunding, monthly_crowdfunding,
        first_available_allocated_mocos, first_available_consuming_mocos
    )

    coinsallocated = 655040 #654860 #670250

    mocperkgCO2 = final_result_row.mocperkgCO2
    initialMP = final_result_row.mocomp

    # Write header and data to CSV
    writedlm("monthly_result.csv", monthly_result_row, ',')
    
end
CSV.write("Monthly Results with Crowdfunding.csv", monthly_result_df)
println(monthly_result_row)

In [1]:
x = 10 # x defines number of working day in a month

coinsallocated = 655040 #670250 #655030 #645860 # 1.1369356986281308e6 # 1.164475393364052e6
allocated_mocos = 0
consuming_mocos = 0

crowdfunding = 0
totalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

totalcardem = 0
totalbusdem = 0
totalbikedem = 0
totaldem = 0

monthly_mocperkgCO2 = 0
last_mocomp = 0
monthly_coins_allocated = 0
monthly_coins_needed = 0
monthly_market_delta = 0
last_cardem = 0
last_busdem = 0
last_bikedem = 0
last_flows_car = 0
last_flows_bus = 0
last_flows_bike = 0
monthly_mocos_car = 0
monthly_mocos_bus = 0
monthly_mocos_bike = 0
monthly_emissions_car = 0
monthly_emissions_bus = 0
monthly_total_tt = 0
monthly_total_tt_c = 0 
monthly_total_tt_b = 0
monthly_total_tt_r = 0
monthly_total_dist_car = 0
monthly_total_dist_bus = 0
monthly_total_dist_rad = 0

mocperkg = [1.0]
        # 2.5,
        # 5.0,
        # 7.5]

monthly_result_row = ()
day_result_row = ()
final_result_row = ()
daily_results_df = DataFrame()
monthly_result_df= DataFrame()

for x in 10:-1:1 # 20 working days in a month
    #for coinsallocated in coinsallocatedarr1
    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        day_result_row = ( 
            coins_allocated = coinsallocatedarr[min_delta_index],
            coins_needed = coinsneededarr[min_delta_index]
        )
    end

    allocated_mocos += day_result_row.coins_allocated
    
    exp_model(x) = 897.27 / (1 + exp(0.22 * (x + 32.69))) # 20 working days in a month
    crowdfunding = exp_model(x) * (655040)
    #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
    totalcrowdfunding += crowdfunding

    coinsallocated = day_result_row.coins_allocated #- crowdfunding

    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        final_result_row = ( 
            mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
            mocomp = MParr[min_delta_index], 
            coins_allocated = coinsallocatedarr[min_delta_index], 
            coins_needed = coinsneededarr[min_delta_index], 
            market_delta = deltaarr[min_delta_index], 
            cardem = cardemarr[min_delta_index], 
            busdem = busdemarr[min_delta_index], 
            bikedem = bikedemarr[min_delta_index], 
            flows_car = flowscararr[min_delta_index], 
            flows_bus = flowsbusarr[min_delta_index], 
            flows_bike = flowsbikearr[min_delta_index], 
            mocos_car = mocoscararr[min_delta_index], 
            mocos_bus = mocosbusarr[min_delta_index], 
            mocos_bike = mocosbikearr[min_delta_index], 
            emissions_car = emissionscararr[min_delta_index], 
            emissions_bus = emissionsbusarr[min_delta_index], 
            total_tt = totaltt[min_delta_index], 
            total_tt_c = totalttc[min_delta_index], 
            total_tt_b = totalttb[min_delta_index], 
            total_tt_r = totalttr[min_delta_index], 
            total_dist_car = totaldcarr[min_delta_index], 
            total_dist_bus = totaldbarr[min_delta_index], 
            total_dist_rad = totaldrarr[min_delta_index]
        )
    end
    # Merge crowdfunding into your daily row
    daily_row_with_crowdfunding = merge(final_result_row, (crowdfunding = crowdfunding,))
    daily_results_df = DataFrame([daily_row_with_crowdfunding])
    push!(monthly_result_df, daily_results_df[1, :])

    consuming_mocos += final_result_row.coins_needed

    available_allocated_mocos = allocated_mocos #+ crowdfunding
    available_consuming_mocos = consuming_mocos #+ crowdfunding

    nextcardem = final_result_row.cardem
    nextbusdem = final_result_row.busdem
    nextbikedem = final_result_row.bikedem
    nextdem = final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem

    totalcardem += final_result_row.cardem
    totalbusdem += final_result_row.busdem
    totalbikedem += final_result_row.bikedem
    totaldem += (final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem)

    monthly_mocperkgCO2 += final_result_row.mocperkgCO2
    last_mocomp = final_result_row.mocomp
    available_monthly_coins_allocated = available_allocated_mocos
    available_monthly_coins_needed = available_consuming_mocos 
    monthly_market_delta += final_result_row.market_delta
    last_cardem = final_result_row.cardem 
    last_busdem = final_result_row.busdem
    last_bikedem = final_result_row.bikedem
    last_flows_car = final_result_row.flows_car
    last_flows_bus = final_result_row.flows_bus
    last_flows_bike = final_result_row.flows_bike
    monthly_mocos_car += final_result_row.mocos_car
    monthly_mocos_bus += final_result_row.mocos_bus
    monthly_mocos_bike += final_result_row.mocos_bike
    monthly_emissions_car += final_result_row.emissions_car
    monthly_emissions_bus += final_result_row.emissions_bus
    monthly_total_tt += final_result_row.total_tt
    monthly_total_tt_c += final_result_row.total_tt_c 
    monthly_total_tt_b += final_result_row.total_tt_b
    monthly_total_tt_r += final_result_row.total_tt_r
    monthly_total_dist_car += final_result_row.total_dist_car
    monthly_total_dist_bus += final_result_row.total_dist_bus
    monthly_total_dist_rad += final_result_row.total_dist_rad
    first_available_allocated_mocos = available_allocated_mocos
    first_available_consuming_mocos = available_consuming_mocos
    first_crowdfunding = crowdfunding
    monthly_crowdfunding = totalcrowdfunding

    monthly_result_row = ( 
        monthly_mocperkgCO2 = monthly_mocperkgCO2, last_mocomp, available_monthly_coins_allocated, available_monthly_coins_needed, monthly_market_delta,
        last_cardem, last_busdem, last_bikedem, 
        last_flows_car, last_flows_bus, last_flows_bike, 
        monthly_mocos_car, monthly_mocos_bus, monthly_mocos_bike, monthly_emissions_car, monthly_emissions_bus, 
        monthly_total_tt, monthly_total_tt_c, monthly_total_tt_b, monthly_total_tt_r, 
        monthly_total_dist_car, monthly_total_dist_bus, monthly_total_dist_rad,
        totalcardem, totalbusdem, totalbikedem, totaldem, first_crowdfunding, monthly_crowdfunding,
        first_available_allocated_mocos, first_available_consuming_mocos
    )

    coinsallocated = 655040 #654860 #670250

    mocperkgCO2 = final_result_row.mocperkgCO2
    initialMP = final_result_row.mocomp

    # Write header and data to CSV
    writedlm("first_one_monthly_result.csv", monthly_result_row, ',')
    
end
CSV.write("First one Monthly Results with Crowdfunding.csv", monthly_result_df)
println(monthly_result_row)

UndefVarError: UndefVarError: DataFrame not defined

In [ ]:
x = 30 # x defines number of working day in a month

coinsallocated = 655040 #670250 #655030 #645860 # 1.1369356986281308e6 # 1.164475393364052e6
allocated_mocos = 0
consuming_mocos = 0

crowdfunding = 0
totalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

totalcardem = 0
totalbusdem = 0
totalbikedem = 0
totaldem = 0

monthly_mocperkgCO2 = 0
last_mocomp = 0
monthly_coins_allocated = 0
monthly_coins_needed = 0
monthly_market_delta = 0
last_cardem = 0
last_busdem = 0
last_bikedem = 0
last_flows_car = 0
last_flows_bus = 0
last_flows_bike = 0
monthly_mocos_car = 0
monthly_mocos_bus = 0
monthly_mocos_bike = 0
monthly_emissions_car = 0
monthly_emissions_bus = 0
monthly_total_tt = 0
monthly_total_tt_c = 0 
monthly_total_tt_b = 0
monthly_total_tt_r = 0
monthly_total_dist_car = 0
monthly_total_dist_bus = 0
monthly_total_dist_rad = 0

mocperkg = [1.0]
        # 2.5,
        # 5.0,
        # 7.5]

monthly_result_row = ()
day_result_row = ()
final_result_row = ()
daily_results_df = DataFrame()
monthly_result_df= DataFrame()

for x in 30:-1:1 # 20 working days in a month
    #for coinsallocated in coinsallocatedarr1
    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        day_result_row = ( 
            coins_allocated = coinsallocatedarr[min_delta_index],
            coins_needed = coinsneededarr[min_delta_index]
        )
    end

    allocated_mocos += day_result_row.coins_allocated
    
    exp_model(x) = 897.27 / (1 + exp(0.22 * (x + 32.69))) # 20 working days in a month
    crowdfunding = exp_model(x) * (655040)
    #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
    totalcrowdfunding += crowdfunding

    coinsallocated = day_result_row.coins_allocated #- crowdfunding

    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        final_result_row = ( 
            mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
            mocomp = MParr[min_delta_index], 
            coins_allocated = coinsallocatedarr[min_delta_index], 
            coins_needed = coinsneededarr[min_delta_index], 
            market_delta = deltaarr[min_delta_index], 
            cardem = cardemarr[min_delta_index], 
            busdem = busdemarr[min_delta_index], 
            bikedem = bikedemarr[min_delta_index], 
            flows_car = flowscararr[min_delta_index], 
            flows_bus = flowsbusarr[min_delta_index], 
            flows_bike = flowsbikearr[min_delta_index], 
            mocos_car = mocoscararr[min_delta_index], 
            mocos_bus = mocosbusarr[min_delta_index], 
            mocos_bike = mocosbikearr[min_delta_index], 
            emissions_car = emissionscararr[min_delta_index], 
            emissions_bus = emissionsbusarr[min_delta_index], 
            total_tt = totaltt[min_delta_index], 
            total_tt_c = totalttc[min_delta_index], 
            total_tt_b = totalttb[min_delta_index], 
            total_tt_r = totalttr[min_delta_index], 
            total_dist_car = totaldcarr[min_delta_index], 
            total_dist_bus = totaldbarr[min_delta_index], 
            total_dist_rad = totaldrarr[min_delta_index]
        )
    end
    # Merge crowdfunding into your daily row
    daily_row_with_crowdfunding = merge(final_result_row, (crowdfunding = crowdfunding,))
    daily_results_df = DataFrame([daily_row_with_crowdfunding])
    push!(monthly_result_df, daily_results_df[1, :])

    consuming_mocos += final_result_row.coins_needed

    available_allocated_mocos = allocated_mocos #+ crowdfunding
    available_consuming_mocos = consuming_mocos #+ crowdfunding

    nextcardem = final_result_row.cardem
    nextbusdem = final_result_row.busdem
    nextbikedem = final_result_row.bikedem
    nextdem = final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem

    totalcardem += final_result_row.cardem
    totalbusdem += final_result_row.busdem
    totalbikedem += final_result_row.bikedem
    totaldem += (final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem)

    monthly_mocperkgCO2 += final_result_row.mocperkgCO2
    last_mocomp = final_result_row.mocomp
    available_monthly_coins_allocated = available_allocated_mocos
    available_monthly_coins_needed = available_consuming_mocos 
    monthly_market_delta += final_result_row.market_delta
    last_cardem = final_result_row.cardem 
    last_busdem = final_result_row.busdem
    last_bikedem = final_result_row.bikedem
    last_flows_car = final_result_row.flows_car
    last_flows_bus = final_result_row.flows_bus
    last_flows_bike = final_result_row.flows_bike
    monthly_mocos_car += final_result_row.mocos_car
    monthly_mocos_bus += final_result_row.mocos_bus
    monthly_mocos_bike += final_result_row.mocos_bike
    monthly_emissions_car += final_result_row.emissions_car
    monthly_emissions_bus += final_result_row.emissions_bus
    monthly_total_tt += final_result_row.total_tt
    monthly_total_tt_c += final_result_row.total_tt_c 
    monthly_total_tt_b += final_result_row.total_tt_b
    monthly_total_tt_r += final_result_row.total_tt_r
    monthly_total_dist_car += final_result_row.total_dist_car
    monthly_total_dist_bus += final_result_row.total_dist_bus
    monthly_total_dist_rad += final_result_row.total_dist_rad
    first_available_allocated_mocos = available_allocated_mocos
    first_available_consuming_mocos = available_consuming_mocos
    first_crowdfunding = crowdfunding
    monthly_crowdfunding = totalcrowdfunding

    monthly_result_row = ( 
        monthly_mocperkgCO2 = monthly_mocperkgCO2, last_mocomp, available_monthly_coins_allocated, available_monthly_coins_needed, monthly_market_delta,
        last_cardem, last_busdem, last_bikedem, 
        last_flows_car, last_flows_bus, last_flows_bike, 
        monthly_mocos_car, monthly_mocos_bus, monthly_mocos_bike, monthly_emissions_car, monthly_emissions_bus, 
        monthly_total_tt, monthly_total_tt_c, monthly_total_tt_b, monthly_total_tt_r, 
        monthly_total_dist_car, monthly_total_dist_bus, monthly_total_dist_rad,
        totalcardem, totalbusdem, totalbikedem, totaldem, first_crowdfunding, monthly_crowdfunding,
        first_available_allocated_mocos, first_available_consuming_mocos
    )

    coinsallocated = 655040 #654860 #670250

    mocperkgCO2 = final_result_row.mocperkgCO2
    initialMP = final_result_row.mocomp

    # Write header and data to CSV
    writedlm("first_two_monthly_result.csv", monthly_result_row, ',')
    
end
CSV.write("First two Monthly Results with Crowdfunding.csv", monthly_result_df)
println(monthly_result_row)